<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/youtuber_%E9%80%90%E5%AD%97%E7%A8%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube
!pip install inlp
!pip install speechrecognition
#@markdown 安裝必要套件

#!pip install you-get

In [ ]:
#@title 輸入 Youtube 網址
import os
from pytube import YouTube

url = "https://www.youtube.com/watch?v=Jp8xnYRhWnw" #@param {type:"string"}

def onProgress(stream, chunk, remains):
    total = stream.filesize
    percent = (total-remains) / total * 100
    print('下載中… {:05.2f}%'.format(percent), end='\r')


# yt.streams.filter().get_highest_resolution().download()

yt=YouTube(url,on_progress_callback=onProgress)
yfilename=yt.streams.filter(type="audio").first().download()

# filename=yt.streams.filter(type="audio").first().default_filename

musicname=yfilename[:yfilename.rfind(".")]
# yt.streams.filter(type="audio").first().download()
print("完成下載： ",yfilename)

# print("完成下載： ",yt.streams.first().download())
# print("轉檔中........")
# os.system('{} -i "{}" -vn -sn -dn "{}.mp3"'.format("ffmpeg",filename, musicname))
# print("完成轉檔： {}.mp3".format(musicname))


# # from google.colab import files
# # files.download("{}.mp4".format(filename[:filename.rfind(".")]))
# from google.colab import files
# files.download(f"{musicname}.mp3")

import os
import shutil
import speech_recognition as sr
import concurrent.futures
import wave
import json
import numpy as np
from inlp.convert import chinese

 

mp3Name= yfilename
 
CutTimeDef = 20 
wav_path='wav' 
txt_path='txt' 
thread_num = 10 

workpath=os.path.dirname(mp3Name)
mp3Name=os.path.basename(mp3Name)
FileName = mp3Name[:mp3Name.rfind(".")]+".wav"
os.chdir(workpath)
chk='y' 
 
def reset_dir(path):
    try:
        os.mkdir(path)
    except Exception:
      if chk=="y":
        shutil.rmtree(path)
        os.mkdir(path)
 
def CutFile(FileName, target_path):
 
    # print("CutFile File Name is ", FileName)
    f = wave.open(FileName, "rb")
    params = f.getparams()    
    nchannels, sampwidth, framerate, nframes = params[:4]
    CutFrameNum = framerate * CutTimeDef
    # 讀取格式資訊
    # 一次性返回所有的WAV檔案的格式資訊，它返回的是一個組元(tuple)：聲道數, 量化位數（byte    單位）, 採
    # 樣頻率, 取樣點數, 壓縮型別, 壓縮型別的描述。wave模組只支援非壓縮的資料，因此可以忽略最後兩個資訊
 
    # print("CutFrameNum=%d" % (CutFrameNum))
    # print("nchannels=%d" % (nchannels))
    # print("sampwidth=%d" % (sampwidth))
    # print("framerate=%d" % (framerate))
    # print("nframes=%d" % (nframes))
 
    str_data = f.readframes(nframes)
    f.close()  # 將波形資料轉換成陣列
    # Cutnum =nframes/framerate/CutTimeDef
    # 需要根據聲道數和量化單位，將讀取的二進位制資料轉換為一個可以計算的陣列
    wave_data = np.frombuffer(str_data, dtype=np.short)
    wave_data.shape = -1, 2
    wave_data = wave_data.T
    temp_data = wave_data.T
    # StepNum = int(nframes/200)
    StepNum = CutFrameNum
    StepTotalNum = 0
    haha = 0
    while StepTotalNum < nframes:
        # for j in range(int(Cutnum)):
        # print("Stemp=%d" % (haha))
        SaveFile = "%s-%03d.wav" % (FileName[:-4], (haha+1))
        # print(FileName)
        if haha % 3==0:
          print("*",end='')
        temp_dataTemp = temp_data[StepNum * (haha):StepNum * (haha + 1)]
        haha = haha + 1
        StepTotalNum = haha * StepNum
        temp_dataTemp.shape = 1, -1
        temp_dataTemp = temp_dataTemp.astype(np.short)  # 開啟WAV文件
        f = wave.open(target_path+"/" + SaveFile, "wb")
        # 配置聲道數、量化位數和取樣頻率
        f.setnchannels(nchannels)
        f.setsampwidth(sampwidth)
        f.setframerate(framerate)
        # 將wav_data轉換為二進位制資料寫入檔案
        f.writeframes(temp_dataTemp.tobytes())
        f.close()
 

 
def texts_to_one(path, target_file):
    files = os.listdir(path)
    files.sort()
    files = [path+"/" + f for f in files if f.endswith(".txt")]
    with open(target_file, "w", encoding="utf-8") as f:
        for file in files:
            with open(file, "r", encoding='utf-8') as f2:
                f.write(f2.read())
    print("完成合併, 檔案位於 %s " % target_file)
 
 
def texts2otr(path, target_file, audio_name, timeperiod):
    template = '''<p><span class="timestamp" data-timestamp="{}.000000">{}</span>{}</p><p><br/></p>
    '''
    files = os.listdir(path)
    files.sort()
    content = ''
    files = [path+"/" + f for f in files if f.endswith(".txt")]
    with open(target_file, "w", encoding="utf-8") as f:
 
        for file in files:
            with open(file, "r", encoding="utf-8") as f2:
                txt = f2.read().split("\n")
                if len(txt) < 2:
                    continue
                pos=txt[0].rfind(".")
                time=int(txt[0][pos-3:pos])
                # times = (int(txt[0].split("-")[1][:-5])-1)*CutTimeDef
                times=(time-1)*CutTimeDef
                secs, mins = times % 60, (times//60) % 60
                hours = (times//60)//60
                timeF = "{:02d}:{:02d}:{:02d}".format(hours, mins, secs)
                content += template.format(times, timeF, txt[1])
 
        output = {"text": content, "media": audio_name,
                  "media-time": timeperiod}
        f.write(json.dumps(output, ensure_ascii=False))
    print("完成合併, otr 檔案位於 %s " % target_file)
   
#@title 執行音頻轉換與分割
 
print(" mp3 轉 wav 檔 ".center(100,'=')) 
os.system('{} -i "{}" "{}"'.format("ffmpeg",mp3Name, FileName))
print(" Wav 檔名為 {} ".format(FileName).center(96))
reset_dir(wav_path)
reset_dir(txt_path)
# # Cut Wave Setting

print(" 音頻以每{}秒分割 ".format(CutTimeDef).center(94,'='))
CutFile(FileName, wav_path)
print("")
print(" 完成分割 ".center(100,'-'))
#@title 執行語音轉文字 (需要耗費不少時間)

#@markdown 指定翻譯的語言類型，如何設定語系請參考 [支援列表](https://cloud.google.com/speech-to-text/docs/languages)
voiceLanguage="cmn-Hant-TW" #@param {type:"string"}

def VoiceToText_thread(file):
  txt_file = "%s/%s.txt" % (txt_path, file[:-4])
      
  if os.path.isfile(txt_file):
    return
  with open("%s/%s.txt" % (txt_path, file[:-4]), "w", encoding="utf-8") as f:
    f.write("%s:\n" % file)
    r = sr.Recognizer()  # 預設辨識英文
    with sr.WavFile(wav_path+"/"+file) as source:  # 讀取wav檔
      audio = r.record(source)
      # r.adjust_for_ambient_noise(source)
      # audio = r.listen(source)
    try:
      text = r.recognize_google(audio,language = voiceLanguage)
      text = chinese.s2t(text)
      # r.recognize_google(audio)
      
      if len(text) == 0:
        print("===無資料==")
        return

      print(f"{file}\t{text}")
      f.write("%s \n\n" % text)
      if file == files[-1]:
          print("結束翻譯")
    except sr.RequestError as e:
      print("無法翻譯{0}".format(e))
      # 兩個 except 是當語音辨識不出來的時候 防呆用的
      # 使用Google的服務
    except LookupError:
      print("Could not understand audio")
    except sr.UnknownValueError:
      print(f"Error: 無法識別 Audio\t {file}")
  



files = os.listdir(wav_path)
files.sort()

with concurrent.futures.ThreadPoolExecutor(max_workers=thread_num) as executor:
    executor.map(VoiceToText_thread, files)
  
# VoiceToText(wav_path, files, txt_path)
 
target_txtfile = "{}.txt".format(FileName[:-4])
texts_to_one(txt_path, target_txtfile)
otr_file = "{}.otr".format(FileName[:-4])
with wave.open(FileName, "rb") as f:
    params = f.getparams()
texts2otr(txt_path, otr_file, FileName, params.nframes)

`

In [14]:
#@title 下載 otr
import os
import shutil
# 搬移音錄檔案到特定的目錄下
source_dir="/content/"
target_dir="/content/drive/MyDrive/tmp/"
def findotr(arr):
  for itm in arr[::-1]:
    if "otr" in itm:
      return itm
otrfilename=findotr(os.listdir("."))
from google.colab import files
files.download("{}".format(otrfilename))
# shutil.move(f"{source_dir}{yfilename}",target_dir)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
#@title 下載 音檔
from google.colab import files
files.download("{}".format(yfilename))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>